<a href="https://colab.research.google.com/github/ducvuuit/CS114.K21.KHTN/blob/master/FinalProjectML/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import các thư viện cần thiết
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

import pandas as pd 



In [2]:
#tải dữ liệu lên
csv_file = open('data-label - 1.csv')


In [3]:
# Kiểm tra dữ liệu
names = ['cmt', 'is_profane']
data = pd.read_csv(csv_file, names = names)
data.head(10)

,cmt,is_profane
0,:( chị đó hỏng chào e,0
1,Zui gần chết,1
2,Zữ qua,0
3,Zô nhà nghỉ đi ỉa hỏi woai,1
4,Yêu thương thế lonn nào được cáii mũi kiaaa :((,1
5,Yêu em,0
6,Yêu đương nhăn nhít,0
7,Yêu chết đi đccc,1
8,Yêu 1 người mà sống bên 1 người ai khóc cho e ...,0
9,Yêu,0


In [4]:
X = data['cmt']

In [5]:
Y = data['is_profane']


In [6]:
Y

0       0
1       1
2       0
3       1
4       1
       ..
3095    0
3096    0
3097    0
3098    1
3099    0
Name: is_profane, Length: 3100, dtype: int64

In [7]:
X

0                                   :( chị đó hỏng chào e
1                                           Zui gần chết 
2                                                  Zữ qua
3                              Zô nhà nghỉ đi ỉa hỏi woai
4         Yêu thương thế lonn nào được cáii mũi kiaaa :((
                              ...                        
3095                                chị dễ thương quá....
3096     chị chấp nhận lời kb emm với emm theo dõi ig ...
3097                                            Chào chị 
3098                                               Cc nhé
3099                          sợ c Tâm bổ vào đầu quá :( 
Name: cmt, Length: 3100, dtype: object

In [8]:
# hàm clean data - loại bỏ các ký tự đặc biệt
import re, string
def clean_text(text):
  if type(text) != str:
    return
  else:
    text = text.lower()
    text = re.sub('\[.*?]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
  return text 

In [9]:
# load stopword
sw =  open('stopword.txt')
data_sw = sw.readlines()
data_sw = [x.strip() for x in data_sw]
print(data_sw)

['a lô', 'a ha', 'ai', 'ai ai', 'ai nấy', 'ai đó', 'alô', 'amen', 'anh', 'anh ấy', 'ba', 'ba ba', 'ba bản', 'ba cùng', 'ba họ', 'ba ngày', 'ba ngôi', 'ba tăng', 'bao giờ', 'bao lâu', 'bao nhiêu', 'bao nả', 'bay biến', 'biết', 'biết bao', 'biết bao nhiêu', 'biết chắc', 'biết chừng nào', 'biết mình', 'biết mấy', 'biết thế', 'biết trước', 'biết việc', 'biết đâu', 'biết đâu chừng', 'biết đâu đấy', 'biết được', 'buổi', 'buổi làm', 'buổi mới', 'buổi ngày', 'buổi sớm', 'bà', 'bà ấy', 'bài', 'bài bác', 'bài bỏ', 'bài cái', 'bác', 'bán', 'bán cấp', 'bán dạ', 'bán thế', 'bây bẩy', 'bây chừ', 'bây giờ', 'bây nhiêu', 'bèn', 'béng', 'bên', 'bên bị', 'bên có', 'bên cạnh', 'bông', 'bước', 'bước khỏi', 'bước tới', 'bước đi', 'bạn', 'bản', 'bản bộ', 'bản riêng', 'bản thân', 'bản ý', 'bất chợt', 'bất cứ', 'bất giác', 'bất kì', 'bất kể', 'bất kỳ', 'bất luận', 'bất ngờ', 'bất nhược', 'bất quá', 'bất quá chỉ', 'bất thình lình', 'bất tử', 'bất đồ', 'bấy', 'bấy chầy', 'bấy chừ', 'bấy giờ', 'bấy lâu', 'bấy lâ

In [10]:
# hàm loại stopword
def del_stopword(text):
    t = clean_text(text)
    if t == '':
        return
    t = t.split()
    t = set(t)
    t = list(t)
    tt = t
    for x in tt:
      if x in data_sw:
        t.remove(x)
    return t

In [11]:
text = 'Đã sai mà cứ ống ống cái mồm ,tao mà như mày tìm cái lỗ cống mà trốn, anh chồng này rất bình tỉnh ,tôi thấy con vợ cứ cải hoài ,gặp phải tôi,'
t = clean_text(text)
t = t.split()
print(t)
t = set(t)
t = list(t)
print(t)
'''tt = t
for x in tt:
    if x in data_sw:
        t.remove(x)
print(t)'''

['đã', 'sai', 'mà', 'cứ', 'ống', 'ống', 'cái', 'mồm', 'tao', 'mà', 'như', 'mày', 'tìm', 'cái', 'lỗ', 'cống', 'mà', 'trốn', 'anh', 'chồng', 'này', 'rất', 'bình', 'tỉnh', 'tôi', 'thấy', 'con', 'vợ', 'cứ', 'cải', 'hoài', 'gặp', 'phải', 'tôi']
['tao', 'vợ', 'tìm', 'thấy', 'bình', 'này', 'lỗ', 'mày', 'trốn', 'đã', 'cứ', 'phải', 'ống', 'cải', 'con', 'mà', 'cái', 'cống', 'anh', 'hoài', 'mồm', 'như', 'gặp', 'tôi', 'tỉnh', 'chồng', 'rất', 'sai']


'tt = t\nfor x in tt:\n    if x in data_sw:\n        t.remove(x)\nprint(t)'

In [12]:
del_stopword('Đã sai mà cứ ống ống cái mồm ,tao mà như mày tìm cái lỗ cống mà trốn, anh chồng này rất bình tỉnh ,tôi thấy con vợ cứ cải hoài ,gặp phải tôi,')

['tao',
 'vợ',
 'thấy',
 'bình',
 'lỗ',
 'mày',
 'trốn',
 'cứ',
 'ống',
 'cải',
 'mà',
 'cống',
 'hoài',
 'mồm',
 'gặp',
 'tỉnh',
 'chồng',
 'sai']

In [13]:
print(len(X))

3100


In [14]:
for i in range(len(X)):
    X[i] = del_stopword(X[i])
    X[i] = ' '.join(X[i])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
X

0                                             chào hỏng e
1                                                zui chết
2                                                      zữ
3                                  woai ỉa zô đi nhà nghỉ
4                      cáii yêu kiaaa thương lonn thế mũi
                              ...                        
3095                                           chị thương
3096    tok lắm mong add lâu chị kb ig dõi tik chấp fb...
3097                                                 chào
3098                                                   cc
3099                                      c tâm đầu sợ bổ
Name: cmt, Length: 3100, dtype: object

In [16]:
model1 = LogisticRegression()
model2 = MultinomialNB()
model3 = LinearSVC()
# split test:25%, train:75%
trainX,testX,trainY,testY = train_test_split(X,Y,test_size=0.25,random_state=42)

# initialize TfidfVectorizer
tf = TfidfVectorizer(ngram_range=(1,2), max_features=50000, min_df=2)

# transform words to vetor
trainX = tf.fit_transform(trainX.values).toarray()
trainVocab = tf.vocabulary_ 
tf = TfidfVectorizer(vocabulary=trainVocab)
testX = tf.fit_transform(testX.values).toarray()
print("[INFO] Used TfidfVectorizer ... ")

model = LogisticRegression()
model.fit(trainX, trainY)
predictions = model.predict(testX)
print(classification_report(testY,predictions))
'''for i in range(2):
    
    if i == 0:
        model = LogisticRegression()
        print("   [INFO] evaluating Logistic Regression...")

    if i == 2:
        model = LinearSVC()
        print("   [INFO] evaluating SVM...")
    
    if i == 1:
        model = MultinomialNB()
        print("   [INFO] evaluating Naive Bayes...")
    
    # train and evaluating 
    model.fit(trainX, trainY)
    predictions = model.predict(testX)
    print(classification_report(testY,predictions))'''

[INFO] Used TfidfVectorizer ... 
              precision    recall  f1-score   support

           0       0.85      0.98      0.91       581
           1       0.90      0.46      0.61       194

    accuracy                           0.85       775
   macro avg       0.87      0.72      0.76       775
weighted avg       0.86      0.85      0.83       775



'for i in range(2):\n    \n    if i == 0:\n        model = LogisticRegression()\n        print("   [INFO] evaluating Logistic Regression...")\n\n    if i == 2:\n        model = LinearSVC()\n        print("   [INFO] evaluating SVM...")\n    \n    if i == 1:\n        model = MultinomialNB()\n        print("   [INFO] evaluating Naive Bayes...")\n    \n    # train and evaluating \n    model.fit(trainX, trainY)\n    predictions = model.predict(testX)\n    print(classification_report(testY,predictions))'

In [35]:

test = 'cái đéo gì thế, vl'
test = clean_text(test)
test = del_stopword(test)
test = ' '.join(test)


test = tf.fit_transform([test])
predict = model.predict(test)

if predict == 1:
  print("câu bình luận thô tục, xúc phạm, cần được xóa")
else:
  print('Câu bình luận bình thường')

câu bình luận thô tục, xúc phạm, cần được xóa
